In [72]:
import pandas as pd
import torch
from torch.utils.data import DataLoader
import math
import torch.nn as nn 

from dataset import TransformerDataset, get_indices_entire_sequence

In [73]:
df = pd.read_csv("covid_data.csv")
df.set_index("timestamp", inplace=True)

# df["FCR_N_PriceEUR2"] = df["FCR_N_PriceEUR"] ** 2

df.head()

,FCR_N_PriceEUR
timestamp,
2017-01-01 00:00:00+00:00,62.13
2017-01-01 01:00:00+00:00,62.08
2017-01-01 02:00:00+00:00,62.07
2017-01-01 03:00:00+00:00,62.01
2017-01-01 04:00:00+00:00,61.90


In [78]:
test_size = 0.1
step_size = 1
input_variables = ["FCR_N_PriceEUR"]
input_size = len(input_variables)
dec_seq_len = 92
enc_seq_len = 153
output_sequence_length = 48
window_size = enc_seq_len + output_sequence_length 
batch_size = 128
batch_first = False

hidden_size = 1

# Remove test data from dataset
training_data = df[:-(round(len(df)*test_size))]

In [79]:
# make list of (start_idx, end_idx) pairs that are used to slice the time series sequence into chunkc. 
# Should be training data indices only
training_indices = get_indices_entire_sequence(data=training_data,
                                               window_size=window_size, 
                                               step_size=step_size)

# make instance of custom dataset class
training_data = TransformerDataset(data=torch.tensor(training_data[input_variables].values).float(),
                                   indices=training_indices,
                                   enc_seq_len=enc_seq_len,
                                   dec_seq_len=dec_seq_len,
                                   target_seq_len=output_sequence_length)

# make dataloader
training_data = DataLoader(training_data, batch_size)

# choose batch and get src, trg's
i, batch = next(enumerate(training_data))
src, trg, trg_y = batch

# permute if batch_first == False
if batch_first == False:
    shape_before = src.shape
    src = src.permute(1, 0, 2)

    shape_before = trg.shape
    trg = trg.permute(1, 0, 2)

# define ffnn for the embedding dimension
encoder_input_layer = nn.Linear(
    in_features=input_size, 
    out_features=hidden_size 
    )

# generate embedding dimension from src
x = encoder_input_layer(src)

From get_src_trg: data size = torch.Size([37248, 1])


In [80]:
print("[batch_size, seq_len, hidden_size]")
src.shape

[batch_size, seq_len, hidden_size]


torch.Size([153, 128, 1])

In [81]:
print("[batch_size, seq_len, hidden_size]")
x.shape

[batch_size, seq_len, hidden_size]


torch.Size([153, 128, 1])

In [82]:
d_model = hidden_size
max_seq_len = 500
dropout = 0.1

position = torch.arange(max_seq_len).unsqueeze(1)
div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))

nndropout = nn.Dropout(p=dropout)  

if batch_first:
    pe = torch.zeros(1, max_seq_len, d_model)
    pe[0, :, 0::2] = torch.sin(position * div_term)
    pe[0, :, 1::2] = torch.cos(position * div_term)
else:
    pe = torch.zeros(max_seq_len, 1, d_model)
    pe[:, 0, 0::2] = torch.sin(position * div_term)
    pe[:, 0, 1::2] = torch.cos(position * div_term)

if batch_first:
    new_x = x + pe[:,:x.size(1)]
else:
    new_x = x + pe[:x.size(0)]

out = nndropout(new_x)

In [83]:
x.shape

torch.Size([153, 128, 1])

In [85]:
pe[:x.size(0)].shape

torch.Size([153, 1, 1])

In [63]:
pe[0, :, :]

tensor([[0., 1.]])

In [66]:
x[0, :, :]

tensor([[-47.1080,  49.9145],
        [-47.0695,  49.8746],
        [-47.0618,  49.8666],
        [-47.0156,  49.8187],
        [-46.9308,  49.7310],
        [-46.7074,  49.4996],
        [-19.0171,  20.8243],
        [-13.8551,  15.4786],
        [-12.3527,  13.9228],
        [-12.1138,  13.6754],
        [-12.2833,  13.8510],
        [-12.3373,  13.9068],
        [-12.1369,  13.6994],
        [-11.4743,  13.0132],
        [-11.4281,  12.9653],
        [-11.6361,  13.1808],
        [-11.6516,  13.1967],
        [-11.8519,  13.4042],
        [-11.4821,  13.0212],
        [-11.4666,  13.0052],
        [-12.5145,  14.0903],
        [-17.3915,  19.1408],
        [-16.8907,  18.6222],
        [-38.1091,  40.5954],
        [-45.3899,  48.1352],
        [-45.1433,  47.8799],
        [-45.1665,  47.9039],
        [-44.7350,  47.4571],
        [-35.2507,  37.6353],
        [-36.3755,  38.8002],
        [-20.7892,  22.6594],
        [-10.9658,  12.4866],
        [-10.9581,  12.4786],
        [-

In [67]:
bx, bpe = torch.broadcast_tensors(x, pe[:x.size(0)])

In [69]:
bx[0, :, :]

tensor([[-47.1080,  49.9145],
        [-47.0695,  49.8746],
        [-47.0618,  49.8666],
        [-47.0156,  49.8187],
        [-46.9308,  49.7310],
        [-46.7074,  49.4996],
        [-19.0171,  20.8243],
        [-13.8551,  15.4786],
        [-12.3527,  13.9228],
        [-12.1138,  13.6754],
        [-12.2833,  13.8510],
        [-12.3373,  13.9068],
        [-12.1369,  13.6994],
        [-11.4743,  13.0132],
        [-11.4281,  12.9653],
        [-11.6361,  13.1808],
        [-11.6516,  13.1967],
        [-11.8519,  13.4042],
        [-11.4821,  13.0212],
        [-11.4666,  13.0052],
        [-12.5145,  14.0903],
        [-17.3915,  19.1408],
        [-16.8907,  18.6222],
        [-38.1091,  40.5954],
        [-45.3899,  48.1352],
        [-45.1433,  47.8799],
        [-45.1665,  47.9039],
        [-44.7350,  47.4571],
        [-35.2507,  37.6353],
        [-36.3755,  38.8002],
        [-20.7892,  22.6594],
        [-10.9658,  12.4866],
        [-10.9581,  12.4786],
        [-

In [70]:
bpe[0, :, :]

tensor([[0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0